In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import random
import time

import sys
import os
import numpy as np
import copy
from tools.tools import save, load, checkdir
from numba import jit

from tools.config import conf

In [9]:
# here \Lambda is a cutoff, also includes Qt corrections from 2406.11647 and It amplitude from 2410.21260
# this one has running coupling

def alpha_s(coupling, s):
    Nc = 3.0
    Nf = 3.0
    beta2 = (11*Nc-2*Nf)/(12*np.pi)
    s0 = 1.0 # log ratio of IR cutoff to \Lambda_{QCD}
    
    if coupling == 'fixed': return 0.25
    elif coupling == 'running': return (np.sqrt(Nc/(2*np.pi))/beta2)*(1/(s0+s))


def get_IC(amp, namp = None, ip = 'a', nsteps=-1):
    if ip == 'a': amp = np.ones((nsteps+1, 1))*np.arange(nsteps+1)
    elif ip == 'b': amp = np.arange(nsteps+1)[:, None]*np.ones(nsteps+1)
    elif ip == 'c': amp += 1
    if namp is None: return amp
    else: 
        namp = amp.reshape(1, nsteps+1, nsteps+1)*np.ones((nsteps+1, 1, 1))
        return amp, namp
        

def evolve(delta, eta_max, Nf=3.0, ICs="ones", coupling="fixed", s0=1.0):
    Nr = 0.25*(Nf/3.0)
    nsteps = round(eta_max/delta)
    d2 = delta**2
    cp = coupling
    
    G = np.zeros((nsteps + 1, nsteps + 1))
    Q = np.zeros((nsteps + 1, nsteps + 1))
    G2 = np.zeros((nsteps + 1, nsteps + 1))
    Qt = np.zeros((nsteps + 1, nsteps + 1))
    GmT = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))
    GmB = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))

    I3t = np.zeros((nsteps + 1, nsteps + 1))
    I3 = np.zeros((nsteps + 1, nsteps + 1))
    I4 = np.zeros((nsteps + 1, nsteps + 1))
    I5 = np.zeros((nsteps + 1, nsteps + 1))
    It = np.zeros((nsteps + 1, nsteps + 1))
    Gm3t = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm3 = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.zeros((nsteps + 1, nsteps + 1, nsteps + 1))


    amps = [G, Q, G2, Qt, I3t, I3, I4, I5, It]
    namps = [GmT, GmB, Gm2, _, Gm3t, Gm3, Gm4, Gm5, _]

    # fixed amplitude
    if isinstance(ICs, list) or isinstance(ICs, np.ndarray):
        if len(ICs) != 7: raise ValueError(f'Length of initial conditions list ({len(ICs)}) is not equal to number of amplitudes (7)')
        for i, amp, ne_amp, ic in enumerate(zip(amps, namps, ICs)):
            amp += 1
            amp *= ic
            if i != 3 and i != 8:
                ne_amp += 1
                ne_amp *= ic

    # basis amp
    elif isinstance(ICs, dict):
        if len(ICs) > 1: raise ValueError(f'Basis amp IC can only take entry at a time')
        
        bamp = list(ICs.keys())[0]
        bvalue = list(ICs.values())[0]

        if bamp == 'G': G, GmT = get_IC(G, GmT, bvalue, nsteps)
        elif bamp == 'Q': Q, GmB = get_IC(Q, GmB, bvalue, nsteps)
        elif bamp == 'G2': G2, Gm2 = get_IC(G2, Gm2, bvalue, nsteps)
        elif bamp == 'Qt': Qt = get_IC(Qt, ip = bvalue, nsteps = nsteps)
        elif bamp == 'I3t': I3t, Gm3t = get_IC(I3t, Gm3t, bvalue, nsteps)
        elif bamp == 'I3': I3, Gm3 = get_IC(I3, Gm3, bvalue, nsteps)
        elif bamp == 'I4': I4, Gm4 = get_IC(I4, Gm4, bvalue, nsteps)
        elif bamp == 'I5': I5, Gm5 = get_IC(I5, Gm5, bvalue, nsteps)
        elif bamp == 'It': It = get_IC(It, ip = bvalue, nsteps = nsteps)

                
    elif ICs == 'ones':
        for i, amp, ne_amp in enumerate(zip(amps, namps)):
            amp += 1
            if i != 3 and i != 8: ne_amp += 1

    else: raise ValueError(f'Initial coniditon ({ICs}) not recognized')
 
    print("amplitudes initialized to", ICs)
    print("Nf=", Nf)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            Q[i, j] = Q[i, j-1]
            Q[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(Q[i:j-1, j-1] + 2*G2[i:j-1, j-1]))
            Q[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 2*G2[:i, j-i:j]))
            Q[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(2*G[i:j-1,j-1] + Q[i:j-1,j-1] + 2*G2[i:j-1,j-1] + 2*GmT[i, i:j-1, j-1] - GmB[i, i:j-1, j-1] + 2*Gm2[i, i:j-1, j-1]))
            
            GmB[i, i, j] = Q[i, j]
            for k in range(i+1, j+1):
                GmB[i, k, j] = GmB[i, k-1, j-1]
                GmB[i, k, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(k-1, j-1))*(Q[k-1:j-1, j-1] + 2*G2[k-1:j-1, j-1]))
                ii = max(i, k-1)
                GmB[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(2*GmT[i, ii:j-1, j-1] - GmB[i, ii:j-1, j-1] + 2*Gm2[i, ii:j-1, j-1] + 2*G[ii:j-1, j-1] + Q[ii:j-1, j-1] + 2*G2[ii:j-1, j-1]))
            
            G[i, j] = G[i, j-1]
            G[i, j] -= Nr*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:i] + 2*G2[:i, j-i:i]))
            G[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(3*G[i:j-1, j-1] + 2*G2[i:j-1, j-1] - 2*Nr*Qt[i:j-1, j-1]))
            G[i, j] += d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(GmT[i, i:j-1, j-1] + (2-2*Nr)*Gm2[i, i:j-1, j-1] - Nr*GmB[i, i:j-1, j-1]))
            
            GmT[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                GmT[i, k, j] = GmT[i, k-1, j-1]
                ii = max(i, k-1)
                GmT[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(GmT[i, ii:j-1, j-1] + (2-2*Nr)*Gm2[i, ii:j-1, j-1] - Nr*GmB[i, ii:j-1, j-1] + 3*G[ii:j-1, j-1] + 2*G2[ii:j-1, j-1] - 2*Nr*Qt[ii:j-1, j-1]))
            
            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(G[:i, j-i:j] + 2*G2[:i, j-i:j]))

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            Qt[i, j] = Qt[i, j-1]
            Qt[i, j] -= d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 2*G2[:i, j-i:j]))
              
            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(4*Gm3t[i, i:j-1, j-1] - 2*Gm3[i, i:j-1, j-1] - 4*Gm4[i, i:j-1, j-1] + 2*Gm5[i, i:j-1, j-1] - 2*Gm2[i, i:j-1, j-1]))
            I3[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(4*I3t[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j]))
            
            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1]
                ii = max(i, k-1)
                Gm3[i, k, j] += d2*np.sum(alpha_s(cp,delta*np.arange(ii, j-1))*(2*Gm3t[i, ii:j-1, j-1] - Gm3[i, ii:j-1, j-1] - 2*Gm4[i, ii:j-1, j-1] + Gm5[i, ii:j-1, j-1] - Gm2[i, ii:j-1, j-1]))
            
            I3t[i, j] = I3t[i, j-1]
            I3t[i, j] += 0.5*d2*np.sum(alpha_s(cp,delta*np.arange(i, j-1))*(2*Gm3t[i, i:j-1, j-1] - 2*Nr*Gm3[i, i:j-1, j-1] + (4*Nr - 4)*Gm4[i, i:j-1, j-1] + (2 - 2*Nr)*Gm5[i, i:j-1, j-1] + (2*Nr - 2)*Gm2[i, i:j-1, j-1]))
            I3t[i, j] += 0.5*d2*np.trace(alpha_s(cp,delta*np.arange(i))[:, np.newaxis]*(-2*Nr*I3[:i, j-i:j] + 4*I3t[:i, j-i:j] + (4*Nr-4)*I4[:i, j-i:j] + (2 - 2*Nr)*I5[:i, j-i:j] - 4*G[:i, j-i:j] + (2*Nr - 6)*G2[:i, j-i:j]))
            
            Gm3t[i, i, j] = I3t[i, j]
            for k in range(i+1, j+1):
                Gm3t[i, k, j] = Gm3t[i, k-1, j-1]
                Gm3t[i, k, j] += d2*np.sum(delta*alpha_s(cp,np.arange(ii, j-1))*(Gm3t[i, ii:j-1, j-1] - Nr*Gm3[i, ii:j-1, j-1] + (2*Nr - 2)*Gm4[i, ii:j-1, j-1] + (1 - Nr)*Gm5[i, ii:j-1, j-1] + (Nr - 1)*Gm2[i, ii:j-1, j-1]))

            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] + G2[:i, j-i:j]))
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(-2*I3t[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 2*G[:i, j-i:j] + 3*G2[:i, j-i:j]))
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(delta*alpha_s(cp,np.arange(i))[:, np.newaxis]*(Q[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j]))
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [Q, G, G2, Qt, I3, I3t, I4, I5, It]

    

In [ ]:
ICs_tag = "basis"
output_dir = "evolved_dipoles"
Nf = 3

eta_max = 15
delta = 0.05
nsteps = round(eta_max/delta)

if nsteps > 800:
    print("Error: nsteps > 800 and will probably crash your computer")
    exit()
    
print("nsteps = ", nsteps)
print('starting (d, eta_max) =', delta, eta_max)

delta_str = "d" + str(delta)[2:] + "_"
amp_strs = {0:"Q", 1:"G", 2:"G2", 3:"Qt", 4:"I3", 5:"I3t", 6:"I4", 7:"I5", 8:"It"}

for iamp in range(9):
    for ibasis in ['a', 'b', 'c']:

        IC = {amp_strs[iamp]: ibasis}
        output = evolve(delta, eta_max, ICs = IC, coupling='running')

        # save output
        if not os.path.exists(output_dir+delta_str+ICs_tag): os.mkdir(output_dir+delta_str+ICs_tag)
        for jamp in range(9):
            np.savetxt(output_dir+delta_str+ICs_tag+"/"+delta_str+"NcNf"+str(Nf)+"_"+amp_strs[iamp]+list(IC.values())[0]+"_"+amp_strs[jamp]+"_rc.dat", output[jamp])
        print("wrote out amplitudes to", output_dir)
        output = 0

